In [27]:
import os
import pandas as pd

from os import listdir
from os.path import isdir

import librosa
import librosa.display

import numpy as np
import struct


In [18]:
current_path = os.getcwd()
current_path = os.getcwd()
genre_music_path = os.path.join(current_path, 'genres_original')
class_xls_path = os.path.join(current_path, 'class_xls')

In [6]:
# Number classification
def GetGenre(genre_music_path):
    label_names = [item for item in os.listdir(
        genre_music_path) if os.path.isdir(os.path.join(genre_music_path, item))]
    nb_train_samples = sum([len(files) for _, _, files in os.walk(genre_music_path)])
    
    return label_names, nb_train_samples

In [7]:
label_names, nb_train_samples = GetGenre(genre_music_path)
nb_classes = len(label_names)
print('Training a CNN Multi-Classifier Model ......')
print('\n - names of classes: ', label_names,
      '\n - # of classes: ', nb_train_samples)

Training a CNN Multi-Classifier Model ......

 - names of classes:  ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'] 
 - # of classes:  1000


In [22]:
def extract_file_info_toCSV(genre_music_path, label_names):
    list_file_names = []

    # list_start = []
    # list_end = []

    list_classIds = []
    list_labels = []

    for idx,label in enumerate(label_names):
        for dirpath, _ ,files in os.walk(os.path.join(genre_music_path, label)):
            for file_name in files:
                list_file_names.append(file_name)
                list_classIds.append(idx)
                list_labels.append(label)
            # data, samplingrate = librosa.load(os.path.join(dirpath, files))

    df = pd.DataFrame({'file_name': list_file_names,
                    'classID': list_classIds,
                    'class': list_labels})
    df.to_csv(os.path.join(class_xls_path,'features_store_01.csv'))


In [23]:
extract_file_info_toCSV(genre_music_path, label_names)

In [25]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

In [31]:
feature = extract_features('D:\\testRGB\MusicClasiffication\genres_original\\blues\\blues.00002.wav')
print(feature, len(feature))

[-90.722595   140.4463     -29.09389     31.684334   -13.984504
  25.764742   -13.664875    11.639933   -11.783643     9.7187605
 -13.133803     5.791199    -8.907628    -1.077       -9.229274
   2.4516892   -7.7290926   -1.8164068   -3.4397204   -2.2312589
  -4.0509806   -0.5722491   -2.018496    -5.3876753   -5.1373224
  -7.8074117   -1.173293    -0.21123998   1.3916391   -3.0024896
  -1.370161    -4.7413645   -0.37558335   0.48872375   0.44240206
  -4.162791    -4.815226    -6.7030134   -4.424409    -0.98426604] 40


In [28]:


class WavFileHelper():
    
    def read_file_properties(self, filename):
        wave_file = open(filename,"rb")
        
        riff = wave_file.read(12)
        fmt = wave_file.read(36)
        
        num_channels_string = fmt[10:12]
        num_channels = struct.unpack('<H', num_channels_string)[0]

        sample_rate_string = fmt[12:16]
        sample_rate = struct.unpack("<I",sample_rate_string)[0]

        bit_depth_string = fmt[22:24]
        bit_depth = struct.unpack("<H",bit_depth_string)[0]
        return (num_channels, sample_rate, bit_depth)

In [30]:
waveHelper = WavFileHelper()
info = waveHelper.read_file_properties('D:\\testRGB\MusicClasiffication\genres_original\\blues\\blues.00002.wav')

print(info)

(1, 22050, 16)
